# Portfolio Hedging (PCA sketch)

In [ ]:
import numpy as np
np.random.seed(0)
X=np.random.randn(1000,5)@np.diag([1,0.5,0.2,0.1,0.1])
C=np.cov(X,rowvar=False); w=np.linalg.eigh(C)[1][:,0]
print('first factor weights (norm):', w/np.linalg.norm(w))